In [1]:
import numpy as np              
from time import time
from os import listdir
from glob import glob
import cv2
import os 
import scipy.linalg as sg
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
from math import sqrt
%matplotlib inline


In [2]:
def dirr_carps(url):
    list_elements_1,list_elements_2=[],[]
    for carp in listdir(url):
        list_elements_1.append(url+carp)
        list_elements_2.append(carp)
    return sorted(list_elements_1),sorted(list_elements_2)

In [3]:
def cambio_clase(str_h):
    cont=0
    for i in str_h:
        if(cont==2):
            num=str(int(i)+1)
        if(i=="_"):
            cont=cont+1
    return num

In [4]:
#dir es la direccion resultante de list_elements
#como output obtiene una cadena de texto con las trayectorias
def tr_text(dir):
    temp=''
    with open(dir) as fp:  
        line = fp.readline()
        cnt = 1
        while line:
            line = fp.readline()
            temp= temp+line[:-1]
            cnt += 1
    return(temp)

In [5]:
def cortar(x):
    #x = trayectorias en texto            
    tr=[]
    gtr=[]
    count = 0
    n = 0
    for i in range(0,len(x)):
        if(x[i]==' '):
            count=count+1
            tr.append(float(x[n:i]))
            n=i
        if(count==34):
            gtr.append(tr)
            count=0
            tr=[]
    return gtr

In [6]:
def flujo_optico(imgBGR ):
    
    tem = cv2.cvtColor(imgBGR,cv2.COLOR_BGR2YCrCb)
    f,c,ch = tem.shape
    imgYcrcb = np.zeros((f,c,ch))
    direction_matrix = np.zeros((f,c))
   
    for row in range(0,tem.shape[0]):         #y
        for col in range(0,tem.shape[1]):     #x
            
            imgYcrcb[row,col,1] = tem[row,col,1] + (-128.0)                 #Cr
            imgYcrcb[row,col,2] = tem[row,col,2] + (-128.0)                 #Cb
            imgYcrcb[row,col,0] = tem[row,col,0]                          #Y
                                  
            if imgYcrcb[row,col,1] > 0 and imgYcrcb[row,col,2] > 0:
                angle = (np.arctan(imgYcrcb[row][col][1]/imgYcrcb[row][col][2])* (180.0/np.pi)) + 270
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] > 0 and imgYcrcb[row,col,2] < 0:
                angle = np.arctan(imgYcrcb[row][col][2]/imgYcrcb[row][col][1])* (180.0/np.pi)
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] < 0 and imgYcrcb[row,col,2] < 0:
                angle = (np.arctan(imgYcrcb[row][col][1]/imgYcrcb[row][col][2])* (180.0/np.pi))+ 90
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] < 0 and imgYcrcb[row,col,2] > 0:
                angle = (np.arctan(imgYcrcb[row][col][2]/imgYcrcb[row][col][1])* (180.0/np.pi)) + 180
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] > 0 and imgYcrcb[row,col,2] == 0:
                angle = 0.0 
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] < 0 and imgYcrcb[row,col,2] == 0:
                angle = 180.0 
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] == 0 and imgYcrcb[row,col,2] > 0:
                angle = 270.0
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] == 0 and imgYcrcb[row,col,2] < 0:
                angle = 90.0 
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] == 0 and imgYcrcb[row,col,2] == 0:
                angle = 0.0 
                direction_matrix[row][col] = angle   
    
    mag,d=cv2.cartToPolar(imgYcrcb[:,:,2],imgYcrcb[:,:,1])    #cartTopolar(x,y)
    return imgYcrcb,mag,abs(direction_matrix) 

In [7]:
def pintar(cov):
    new_cov=np.copy(cov)
    for i in range(cov.shape[1]):
        for j in range(cov.shape[0]):
            if(new_cov[i,j]<=0):
                new_cov[i,j]=-1*(new_cov[i,j]/np.min(cov))
            else:
                new_cov[i,j]=new_cov[i,j]/np.max(cov)
    return new_cov

In [8]:
def normalized_matrices(covs):
    d=covs[0][0].shape[0]
    min_matrix=np.zeros((d,d))
    max_matrix=np.zeros((d,d))
    for i in range(len(covs)):
        #for de frames
        for j in range(len(covs[i])):
            #for de trayectorias en los frames
            new_cov=covs[i][j]
            for k in range(covs[i][j].shape[0]):
                for l in range(covs[i][j].shape[1]):
                    #for's para las dimensiones de las covarianzas(# de primitivas)
                    if(new_cov[k,l]<min_matrix[k,l]):
                        min_matrix[k,l]=new_cov[k,l]
                    if(new_cov[k,l]>max_matrix[k,l]):
                        max_matrix[k,l]=new_cov[k,l]
    return max_matrix,min_matrix

In [9]:
def normalized_draw(cov,max_matrix,min_matrix):
    new_cov=np.copy(cov)
    for i in range(cov.shape[1]):
        for j in range(cov.shape[0]):
            if(new_cov[i,j]<=0):
                new_cov[i,j]=-1*(new_cov[i,j]/min_matrix[i,j])
            else:
                new_cov[i,j]=new_cov[i,j]/max_matrix[i,j]
    return new_cov

In [13]:
def int_great_array(img,imgf):
# nuestrass 4 caracteristicas de la imagen, puede tenerse en cuenta el color

    gris=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sobelx=cv2.Sobel(gris,cv2.CV_64F,1,0,ksize=3)
    sobely=cv2.Sobel(gris,cv2.CV_64F,0,1,ksize=3)
    sobelyx=((cv2.Sobel(sobelx,cv2.CV_64F,1,0,ksize=3)+cv2.Sobel(sobely,cv2.CV_64F,0,1,ksize=3))/2)
    laplacian=cv2.Laplacian(gris,cv2.CV_64F,ksize=3)
    imgyrcv,mag,direction=flujo_optico(imgf)
    cr=imgyrcv[:,:,1]
    cb=imgyrcv[:,:,2]
    #sobelx=cv2.resize(sobelx, (mag.shape[1],mag.shape[0]))
    #sobely=cv2.resize(sobely, (mag.shape[1],mag.shape[0]))
    #sobelyx=cv2.resize(sobelyx, (mag.shape[1],mag.shape[0]))
    #laplacian=cv2.resize(laplacian, (mag.shape[1],mag.shape[0]))
    ch_array=np.array([sobelx,sobely,sobelyx,laplacian,cr,cb,mag])
    #print(sobelx.shape,sobely.shape,sobelyx.shape,laplacian.shape,cr.shape,cb.shape,mag.shape)
    #Matriz de d*w*h
    #print(ch_array,ch_array.shape)
    d=ch_array.shape[0]
    W=ch_array.shape[1]
    H=ch_array.shape[2]
    #Tensor P
    f_mat=np.zeros((d,W+1,H+1))
    for i in range(d):
        img_int=cv2.integral(ch_array[i],sdepth=cv2.CV_64F)
        f_mat[i]=img_int 
    #Tensor Q    
    nmatrices=((d * (d + 1)) / 2 )
    matrix_product=np.zeros((int(nmatrices),W+1,H+1))              
    pos=0
    for i in range(d):
        for j in range(i,d):
            mult=ch_array[i]*ch_array[j]                      #producto elemento a elemento entre tipos arrays
            img_int=cv2.integral(mult,sdepth=cv2.CV_64F)
            matrix_product[pos] = img_int
            pos=pos+1
        

    return ch_array,f_mat,matrix_product

In [14]:
def matriz_covarianza_x16(Px16,Qx16,T_area):
    #El tensor P
    #El tensor Q
    #Las coordenadas de los puntos para las regiones de interes
    d=Px16[0].shape[0]
    m_cov=np.zeros((d,d))
    pos=0
    p_1=0
    p_2=0
    q=0
    
    for i in range(d):
        for j in range(i,d):
            for k in range(len(Qx16)):
                y1=T_area[k][0]
                x1=T_area[k][1]
                y2=T_area[k][2]
                x2=T_area[k][3]
                col=x2-x1
                fil=y2=-y1
                
                q=Qx16[k][pos][y1,x1]+Qx16[k][pos][y1+fil,x1+col]-Qx16[k][pos][y1,x1+col]-Qx16[k][pos][y1+fil,x1]+q
                p_1=(Px16[k][i][y1,x1]+Px16[k][i][y1+fil,x1+col]-Px16[k][i][y1,x1+col]-Px16[k][i][fil+y1,x1])/(fil*col)+p_1
                p_2=(Px16[k][j][y1,x1]+Px16[k][j][y1+fil,x1+col]-Px16[k][j][y1,x1+col]-Px16[k][j][fil+y1,x1])/(fil*col)+p_2
                #print(k)
            n1=(1/((fil*col)-1))*16
            n2=(1/(fil*col))*16    
            m_cov[i,j]=(n1)*(q-(n2*p_1*p_2))
            pos=pos+1
            
            if(i!=j):
                m_cov[j,i]=m_cov[i,j]

                
    return m_cov

In [21]:
def video_covs_x16(path,trayectorias,area,path2):
    #tr_list = mi lista de trayectorias 
    #En el tensor P obtengo listas de d*w*h
    #En el tensor Q obtengo listas de n*w*h, donde n es el numero de elementos de la diagonal superior
    #tr_matrix es una matriz de trayectorias con dimension [nro de trayectorias][nro de frames*2] (x, y)
    #list_cov es una lista con todas mis covarianzas, donde tiene [nro del frame-15][nro de covarianzas por frame]
    #Obtengo como salidas, una lista de listas con todas mis covarianzas, y una matriz de trayectorias, de donde las puedo extraer
    #Para ejecutar esta funcion debo tener antes, mis caracteristicas integrales int_great_array
    
    flujo,f,great_p,great_q,Q=[],[],[],[],[]
    images = [path+image for image in os.listdir(path)]
    n_images = len(images)

    if isinstance(sqrt(n_images)%1,float):limits = int(sqrt(n_images))+1 
    else:limits = int(sqrt(n_images))  
    for i,image in enumerate(images):
        im = Image.open(image)
        a='FramesTemp/frame'+str(i+1)+'.jpg'
        if(i<10):
            zeros='0000000'
            b='FramesF/'+path2+'/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
        elif(i<100):
            zeros='000000'
            b='FramesF/'+path2+'/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
        else:
            zeros='00000'
            b='FramesF/'+path2+'/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
            
        f.append(a)
        
        try: 
            #print(cv2.imread(flujo[i]).shape)
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i]))
        except:
           #xf=cv2.imread(flujo[i-1])
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i-1]))
                                  
        great_p.append(p)
        great_q.append(q)
    #Obtengo el tensor P y Q
    
    d=great_p[0].shape[0]
    tr_list=cortar(trayectorias)
    #nro numero de trayectorias del video
    nro=len(tr_list)
    #nro2 numero de frames del video*2
    nro2=n_images*2
    tr_matrix=np.zeros((nro,nro2))
    x,y=tr_matrix.shape
    
    for i in range(0,x):
        k=2
        for j in range(int((tr_list[i][0]*2)),int((tr_list[i][1]*2))):
            tr_matrix[i][j]=tr_list[i][k]
            k=k+1
    #Obtengo mi matriz de trayectorias segun el caso
    
    #El 32 debido a que empieza en el frame 16 y termina 16 frames antes del final.
    list_cov=[]
    for i in range(nro):
        #iteran trayectorias
        #cov_fr=[]
        Qx16=[]
        Px16=[]
        T_area=[]
        for j in range(32,nro2,2):
            if(np.float32(tr_matrix[i][j])==0):
                pass
            else:
                temp=int(j/2)
                temp_area=[]
            #flag = True
            #if(tr_matrix[i][j]+area < great_p[0].shape[1] ):
                y1=int(tr_matrix[i][j]+area)
            #else:
            #    flag=False
            #if(tr_matrix[i][j]-area > 0 ):
                y2=int(tr_matrix[i][j]-area)
            #else:
            #    flag=False
            #if(tr_matrix[i][j+1]+area > great_p[0].shape[2] ):    
                x1=int(tr_matrix[i][j+1]+area)
            #else:
            #    flag=False                
            #if(tr_matrix[i][j+1]-area > 0 ):
                x2=int(tr_matrix[i][j+1]-area)
                #else:
                #    flag=False
                #if(flag):
                Q=great_q[temp]
                Qx16.append(Q)
                P=great_p[temp]
                Px16.append(P)
                temp_area.append(y1)
                temp_area.append(x1)
                temp_area.append(y2)
                temp_area.append(x2)
                T_area.append(temp_area)

        #print(len(T_area))
        if(len(T_area) > 14  ):     
            try: 
                cov=matriz_covarianza_x16(Px16,Qx16,T_area) 
                list_cov.append(cov)
            except Exception as e:
                e
                
           
    #Obtengo list_cov
    
    return list_cov,tr_matrix,great_p,great_q

In [ ]:
list_elements1,list_elements2=dirr_carps('Datasets/exp01/')
trayectorias=tr_text(list_elements1[17])
#trayectorias=cortar(trayectorias)
area=20
path='Trayectorias_densas/frames2/'

In [ ]:
len(list_elements1)
len(list_elements2)

### Con esto borro los frames de la carpeta, de manera que voy a ir salvando espacio

In [ ]:
import os, shutil 
folder = path 
for the_file in os.listdir(folder): 
    file_path = os.path.join(folder, the_file) 
    try: 
        if os.path.isfile(file_path): 
            os.unlink(file_path) 
     #elif os.path.isdir(file_path): shutil.rmtree(file_path) 
    except Exception as e: 
        print(e)

### Aca se extraen los frames de los videos, tener en cuenta, debe ser automatizado

In [ ]:
vidcap = cv2.VideoCapture('Datasets/datasets/ut/segmented_set1/'+list_elements2[17][:-4]+'.avi')
success,image = vidcap.read()
count = 1
success = True
while success:
    cv2.imwrite("Trayectorias_densas/frames2/frame%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    #print('Read a new frame: ', success)
    count += 1

In [ ]:
'Datasets/datasets/ut/segmented_set1/'+list_elements2[17][:-4]+'.avi'

In [ ]:
list_elements2[17][:-4]
'Datasets/datasets/ut/segmented_set1/'+list_elements2[17][:-4]+'.avi'

In [ ]:
path

In [ ]:
covs_,matrix_,great_p_,great_q_=video_covs_x16(path,trayectorias,area)

In [ ]:
print(len(covs_))
print(matrix_.shape)
print(len(great_p_))
print(len(great_q_))
print(great_p_[0].shape)
print(covs_[0][0].shape[0])
print(great_q_[0].shape)

In [ ]:
test=covs_
np.save('binarycovs/test',test)

In [ ]:
tempo=np.load('test.npy')

In [ ]:
print(len(tempo))
print(tempo[0][0].shape[0])


## LISTO PARA CORRER

In [22]:
import os, shutil 
from tqdm import tqdm
list_elements1,list_elements2=dirr_carps('Datasets/exp01/')
area=20
path='FramesTemp/'

In [23]:
trayectorias=tr_text(list_elements1[1])
#trayectorias

In [24]:
path2=list_elements2[1][:-4]
path2

'10_2_1'

In [30]:
list_elements2[59][:-4]

'9_2_2'

In [ ]:
covs,matrix,great_p,great_q=video_covs_x16(path,trayectorias,area,path2)
len(covs)

In [28]:
for i in range(57,len(list_elements1)):
    print(i)

57
58
59


In [32]:
import gc
gc.collect()

7937

In [31]:
del(covs)
del(matrix)
del(great_p)
del(great_q)

In [33]:

for i in tqdm(range(57,len(list_elements1))):
    trayectorias=tr_text(list_elements1[i])
    path2=list_elements2[i][:-4]
    #print(path)
    #Eliminando frames de la carpeta 
    folder = path 
    for the_file in os.listdir(folder): 
        file_path = os.path.join(folder, the_file) 
        try: 
            if os.path.isfile(file_path): 
                os.unlink(file_path) 
         #elif os.path.isdir(file_path): shutil.rmtree(file_path) 
        except Exception as e: 
            print(e)
    #print('flag')
    
    #Obteniendo frames        
    print(list_elements2[i][:-4])
    
    vidcap = cv2.VideoCapture('Datasets/datasets/ut/segmented_set1/'+list_elements2[i][:-4]+'.avi')
    success,image = vidcap.read()
    count = 1
    success = True
    while success:
        cv2.imwrite("FramesTemp/frame%d.jpg" % count, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
        
    
    #Extrayendo covarianzas    
    covs,matrix,great_p,great_q=video_covs_x16(path,trayectorias,area,path2)
    np.save('binarycovs/'+ list_elements2[i][:-4],covs)
    


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

7_2_5



 33%|███████████████████████████▋                                                       | 1/3 [02:32<05:04, 152.41s/it]

8_2_0



 67%|███████████████████████████████████████████████████████▎                           | 2/3 [05:54<02:47, 167.38s/it]

9_2_2



100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [09:19<00:00, 178.58s/it]

In [ ]:
covs,matrix,great_p,great_q=video_covs_x16(path,trayectorias,area)

In [ ]:
list_elements2[0][:-4]